## Version 0.9.9.4 2021 03 14

### Tbot - Trade Bot

 Tbot is a Robot of trade developed to trade cryptourrencies or s&p500 stocks, based on https/json queries. With few adjustments it may help traders to improve their workflow. In this file we shall compile three steps followed at the "Algorithimic Trading Using Python" course created by the api developer and completed by the developer of this api. ("https://freecodecamp.org")
 The code is also hosted at https://github.com/n4rus/tbot and is licensed under creative commons.
 
### Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this api. 

In [1]:
import numpy as np
import pandas as pd
import requests
import secrets
import xlsxwriter
import math
import datetime
import time

### API token, secret key and acess keys
API tokens (and other sensitive information) shall be stored in a secrets.py file that doesn't get pushed to your local Git repository. The AcessKey and SecretKey are provided by the server admin.

In [2]:
from secrets import SecretKey
from secrets import AccessKey
from novadax import RequestClient as NovaClient
from datetime import datetime
from datetime import date
from time import time

nova_client = NovaClient(AccessKey, SecretKey)

### Importing List of currencies, making the API Call and the table Dataframe
List of crypto-currencies, get balance, and table creation

In [3]:
balance = nova_client.get_account_balance()
bdata = balance['data']

bcolumns = ['Currency', 'Balance']
bdataframe = pd.DataFrame(columns = bcolumns,)
bs = 0
bcr = {}

for bsymbol in bdata:
    pdis = bdata[bs]
    bis = pdis['balance']
    bic = pdis['currency']
    bcr[f"{bic}"] = f"{bis}"   
    bdataframe.append(
        pd.Series([
            bic,
            bis,
        ], index = bcolumns),
        ignore_index = True)
    bdataframe = bdataframe.append(
    pd.Series([
        bic,
        bis,
    ], index=bcolumns),
        ignore_index=True)
    bs = bs + 1  

sublcq = nova_client.subs()
sublcd = sublcq['data']

blccolumns = ['Currency', 'Sub-Balance']
blcdataframe = pd.DataFrame(columns = bcolumns,)

p = 0
subids = {}
sublcs = {}

for i in sublcd:
    sublcd = sublcq['data']
    sub = sublcd[p]
    sublci = sub['subId']
    sublcy = sub['subIdentify']
    subids[f"{sublcy}"] = f"{sublci}"
    subId = sublci
    sublcid = nova_client.subs_balance(subId)
    subd = sublcid['data']
    sd = subd[p]
    subcy = sd['currency']
    sublc = sd['balance']
    sublcs[f"{subcy}"] = f"{sublc}"
    
    blcdataframe.append(
        pd.Series([
            subcy,
            sublc,
        ], index = blccolumns),
        ignore_index = True)
    blcdataframe = blcdataframe.append(
    pd.Series([
        subcy,
        sublc,
    ], index=blccolumns),
        ignore_index=True)
 
    p = p + 1

blcdataframe.sort_values('Balance', ascending = False, inplace = True)
blcdataframe.reset_index(drop = True, inplace = True)
#blcdataframe

bdataframe.sort_values('Balance', ascending = False, inplace = True)
bdataframe.reset_index(drop = True, inplace = True)
#bdataframe
#bdataframe, blcdataframe

### Symbol position, Dolar price and timestamp
For the execution to be done independently for each symbol, the below cell is needed:

In [4]:
symbol = 'ETH_USDT'
symbols = ['ETH_USDT']
currency = 'R$'
my_columns = ['Symbol', 'Ask','Bid', 'Last Price','AVG Price', 'Shares to Buy','High 24h', 'Min 24h', 'Avg 24h', 'Timestamp', 'Requisition Timestamp']
e_dataframe = pd.DataFrame(columns = my_columns)


api_url = f'https://api.novadax.com/v1/market/ticker?symbol={symbol}'
fdaty = requests.get(api_url).json()
y = 0
for a in api_url[:1]:
    S = 0
    fS = str(S)
    for i in api_url:
        fdata = fdaty['data']
        price = float(fdata['lastPrice'])
        ask = float(fdata['ask'])
        bid = float(fdata['bid'])
        edolar = bcr['USDT']
        eth_wal = float(bcr['ETH']) * float(price)
        edolarwal = float(bcr['USDT']) + float(eth_wal)
        shares_to_buy = (float(edolarwal)/float(price))
        high = float(fdata['high24h'])
        mini = float(fdata['low24h'])
        mid = float((high - mini)/2) 
        abl = float((ask + bid + price) / 3)
    
        etsp = (fdata['timestamp'])
        tspmi = float(etsp/1000)
        tspm = datetime.fromtimestamp(tspmi).strftime(f"%Y%m%d %H:%M:S")
    
        localtimestamp = int(time())
        dl = localtimestamp
        tserver = datetime.fromtimestamp(dl).strftime(f"%Y%m%d %H:%M:S")
        
        y = y + 1
    e_dataframe.append(
                pd.Series(
                    [
                        symbol,
                        ask,
                        bid,
                        price,
                        abl,
                        shares_to_buy,
                        high,
                        mid,
                        mini,
                        tspm,
                        tserver
                    ]
                    ,index = my_columns),
                ignore_index=True)
    
       
    e_dataframe = e_dataframe.append(
            pd.Series([
                        symbol, 
                        ask,
                        bid,
                        price,
                        abl,
                        shares_to_buy,
                        high,
                        mid,
                        mini,
                        tspm,
                        tserver
                        ],
                    index = my_columns
                    ),
                    ignore_index = True)

e_dataframe

,Symbol,Ask,Bid,Last Price,AVG Price,Shares to Buy,High 24h,Min 24h,Avg 24h,Timestamp,Requisition Timestamp
0,ETH_USDT,1925.77,1917.97,1915.0,1919.58,0.067797,1915.0,92.925,1729.15,20210313 16:40:S,20210313 16:40:S


In [5]:
symbol = 'ETH_USDT'
symbols = ['ETH_USDT']
currency = 'R$'
my_columns = ['Symbol','Side', 'Last Price', 'Shares to Buy', 'Timestamp', 'Requisition Timestamp']
s_dataframe = pd.DataFrame(columns = my_columns)

api_url = f'https://api.novadax.com/v1/market/ticker?symbol={symbol}'
rdaty = requests.get(api_url).json()
rdat = nova_client.list_trades(symbol, limit = 10)

for a in rdat:
    yz = int(0)
    for i in rdat['data']:
        daty = rdat['data']
        data = daty[yz]
        price = float(data['price'])
        side = data['side']
        edolar = bcr['USDT']
        eth_wal = float(bcr['ETH']) * float(price)
        edolarwal = float(bcr['USDT']) + float(eth_wal)
        shares_to_buy = (float(edolarwal)/float(price))
    
        etsp = (data['timestamp'])
        tspmi = float(etsp/1000)
        tspm = datetime.fromtimestamp(tspmi).strftime("%Y%m%d %H:%M:%S")
    
        localtimestamp = int(time())
        dl = localtimestamp
        tserver = datetime.fromtimestamp(dl).strftime("%Y%m%d %H:%M:%S") 
        yz = yz + 1
    s_dataframe.append(
                pd.Series(
                    [
                        symbol,
                        side,
                        price,
                        shares_to_buy,
                        tspm,
                        tserver
                    ]
                    ,index = my_columns),
                ignore_index=True)
    
       
    s_dataframe = s_dataframe.append(
            pd.Series([
                        symbol, 
                        side,
                        price,
                        shares_to_buy,
                        tspm,
                        tserver
                        ],
                    index = my_columns
                    ),
                    ignore_index = True)

s_dataframe

,Symbol,Side,Last Price,Shares to Buy,Timestamp,Requisition Timestamp
0,ETH_USDT,BUY,1900.0,0.068332,20210313 14:51:19,20210313 16:41:03
1,ETH_USDT,BUY,1900.0,0.068332,20210313 14:51:19,20210313 16:41:03
2,ETH_USDT,BUY,1900.0,0.068332,20210313 14:51:19,20210313 16:41:03


### Creating the momentum table
For mommentum we use the last order minus the order from an amount x of time. The place from wich the data comes is main importance on the formula. As stated in the comment section on the first versions, data must come from the common filled orders or the last trades from the users history, here the second case is uded, as well from this version on, but the first ones used common filled orders.

In [6]:
listtrades = nova_client.list_orders(symbol, 'FINISHED', limit=5)
ldata = listtrades['data']
lindice= ldata[0]
amount = lindice['amount']
my_columns = ['Symbol', 'B/S', 'AVG Price', 'Momentum AVG Price', 'Momentum','Ask', 'Bid', 'Last Price', 'Ask Bid Last AVG', 'Shares to Buy','Min 24h', 'Avg 24h', 'High 24h', 'Timestamp', 'Requisition Timestamp']
m_dataframe = pd.DataFrame(columns = my_columns)
api_url = nova_client.get_ticker(symbol)
rdat = nova_client.list_trades(symbol, limit = 10)

for symbol in symbols:
    listtrades = nova_client.list_orders(symbol, 'FINISHED')
    s = 0
    for i in listtrades['data'][:9]:
        ldata = listtrades['data']
        ldc = ldata[s]
        amount = ldc['amount']
        price = float(ldc['price'])
        
        amount = ldc['amount']
        cprice = float(amount) * float(price)
        edolar = bcr['USDT']
        eth_wal = float(bcr['ETH']) * float(price)
        edolarwal = float(bcr['USDT']) + float(eth_wal)
        sharestobuy = float(amount) - (float(edolarwal))/float(price)
        bs = ldc['side']
        order = float(amount) - float(sharestobuy)
        srto = float(order)
        
        fdaty = api_url
        fdt = fdaty['data']
        fdata = fdt
        price = fdata['lastPrice']
        bid = fdata['bid']
        ask = fdata['ask']
        high = float(fdata['high24h'])
        mini = float(fdata['low24h'])
        mid = float((high - mini)/2) 
        abl = float((float(ask) + float(bid) + float(price)) / 3)
        
        V = price
        vx = float(data['price'])
        M = float(V) - float(vx)
        momentum = float(vx) - float(price)
        tspmi = float(ldc['timestamp']/1000)
        tspm = datetime.fromtimestamp(tspmi).strftime("%Y%m%d %H:%M:%S")
        status = ldc['status']
        
        localtimestamp = int(time())
        dl = localtimestamp
        tserver = datetime.fromtimestamp(dl).strftime("%Y%m%d %H:%M:%S")       
        p = p + 1
        
        s = s + 1
        m_dataframe.append(
                    pd.Series(
                    [
                        symbol, 
                        bs,
                        vx,
                        momentum,
                        M,
                        ask,
                        bid,
                        price,
                        abl,
                        sharestobuy,
                        high,
                        mid,
                        mini,
                        tspm,
                        tserver    
                    ]
                    ,index = my_columns),
                ignore_index=True)

    
        m_dataframe = m_dataframe.append(
                pd.Series([
                        symbol, 
                        bs,
                        vx,
                        momentum,
                        M,
                        ask,
                        bid,
                        price,
                        abl,
                        sharestobuy,
                        high,
                        mid,
                        mini,
                        tspm,
                        tserver                 
                      ]
                ,index = my_columns), 
                ignore_index = True)

m_dataframe.sort_values('Timestamp', ascending = True, inplace = True)
m_dataframe.reset_index(drop = True, inplace = True)
m_dataframe

,Symbol,B/S,AVG Price,Momentum AVG Price,Momentum,Ask,Bid,Last Price,Ask Bid Last AVG,Shares to Buy,Min 24h,Avg 24h,High 24h,Timestamp,Requisition Timestamp
0,ETH_USDT,SELL,1900.0,-15.0,15.0,1927.69,1917.57,1915,1920.086667,-0.031809,1915.0,92.925,1729.15,20210310 13:22:47,20210313 16:41:10
1,ETH_USDT,SELL,1900.0,-15.0,15.0,1927.69,1917.57,1915,1920.086667,-0.031047,1915.0,92.925,1729.15,20210310 16:17:31,20210313 16:41:10
2,ETH_USDT,BUY,1900.0,-15.0,15.0,1927.69,1917.57,1915,1920.086667,-0.002250,1915.0,92.925,1729.15,20210310 17:46:17,20210313 16:41:10
3,ETH_USDT,SELL,1900.0,-15.0,15.0,1927.69,1917.57,1915,1920.086667,-0.000112,1915.0,92.925,1729.15,20210310 20:18:52,20210313 16:41:10
4,ETH_USDT,SELL,1900.0,-15.0,15.0,1927.69,1917.57,1915,1920.086667,-0.001442,1915.0,92.925,1729.15,20210311 11:34:00,20210313 16:41:10
5,ETH_USDT,SELL,1900.0,-15.0,15.0,1927.69,1917.57,1915,1920.086667,-0.000676,1915.0,92.925,1729.15,20210311 11:34:16,20210313 16:41:10
6,ETH_USDT,SELL,1900.0,-15.0,15.0,1927.69,1917.57,1915,1920.086667,-0.001428,1915.0,92.925,1729.15,20210311 12:35:09,20210313 16:41:10
7,ETH_USDT,BUY,1900.0,-15.0,15.0,1927.69,1917.57,1915,1920.086667,-0.001468,1915.0,92.925,1729.15,20210312 15:50:02,20210313 16:41:10
8,ETH_USDT,SELL,1900.0,-15.0,15.0,1927.69,1917.57,1915,1920.086667,-0.000032,1915.0,92.925,1729.15,20210312 15:51:05,20210313 16:41:10


In [7]:
#orders dataframe
ttrades = nova_client.list_trades(symbol, limit=9)
tdata = ttrades['data']
my_columns = ['Symbol', 'Amount', 'Ticker Price', 'B/S', 'Momentum Price',  'momentum-raw-data', 'Status', 'Timestamp', 'Requisition Timestamp']
o_dataframe = pd.DataFrame(columns = my_columns)
api_url = f'https://api.novadax.com/v1/market/ticker?symbol={symbol}'
fdaty = requests.get(api_url).json()
fdat = nova_client.list_trades('BTC_BRL', limit = 10)

for symbol in symbols:
    #from 'finished' to 'filled'
    listorders = nova_client.list_orders(symbol, 'FILLED')
    s = 0
    for i in listorders['data'][:9]:
        ldata = listorders['data']
        ldc = ldata[s]
        amount = ldc['amount']
        price = float(ldc['price'])
        
        amount = float(ldc['amount'])
        cprice = float(amount) * float(price)
        edolar = bcr['USDT']
        eth_wal = float(bcr['ETH']) * float(price)
        edolarwal = float(bcr['USDT']) + float(eth_wal)
        bs = ldc['side']
        
        #rework into a precise formula with momentum
        #This must free the mommentum table cell
        vxdata = tdata[s]
        vxi = vxdata['price']
        momentumprice = float(vx)
        momentumraw = float(price) - float(vxi)
        
        V = price
        vx = float(data['price'])
        M = float(V) - float(vx)
        momentum = float(vx) - float(price)

        tspi = float(ldc['timestamp']/1000)
        tsp = datetime.fromtimestamp(tspi).strftime("%Y%m%d %H:%M:%S")
        status = ldc['status']
        
        localtimestamp = int(time())
        dl = localtimestamp
        tserver = datetime.fromtimestamp(dl).strftime("%Y%m%d %H:%M:%S")

        s = s + 1
        
        o_dataframe.append(
                    pd.Series(
                    [
                        symbol,
                        amount,
                        price,
                        bs,
                        momentumprice,
                        momentumraw,
                        status,
                        tsp,
                        tserver,
                    ]
                    ,index = my_columns),
                ignore_index=True)

    
        o_dataframe = o_dataframe.append(
                pd.Series([
                        symbol,
                        amount,
                        price,
                        bs,
                        momentumprice,
                        momentumraw,
                        status,
                        tsp,
                        tserver,
                    
                      ]
                ,index = my_columns), 
                ignore_index = True)

o_dataframe.sort_values('Timestamp', ascending = False, inplace = True)
o_dataframe.reset_index(drop = True, inplace = True)
o_dataframe

,Symbol,Amount,Ticker Price,B/S,Momentum Price,momentum-raw-data,Status,Timestamp,Requisition Timestamp
0,ETH_USDT,0.0713,1820.00,SELL,1900.0,-95.00,FILLED,20210312 15:51:05,20210313 16:41:17
1,ETH_USDT,0.0714,1781.58,BUY,1900.0,-125.74,FILLED,20210312 15:50:02,20210313 16:41:17
2,ETH_USDT,0.0695,1830.37,SELL,1900.0,-77.93,FILLED,20210311 12:35:09,20210313 16:41:17
3,ETH_USDT,0.0697,1804.33,BUY,1900.0,-105.70,FILLED,20210310 17:46:17,20210313 16:41:17
4,ETH_USDT,0.0391,1850.77,SELL,1900.0,-50.87,FILLED,20210310 16:17:31,20210313 16:41:17
5,ETH_USDT,0.1405,1840.00,SELL,1900.0,-68.43,FILLED,20210309 11:50:06,20210313 16:41:17
6,ETH_USDT,0.1406,1825.00,BUY,1900.0,-85.00,FILLED,20210308 23:10:58,20210313 16:41:17
7,ETH_USDT,0.1413,1816.89,SELL,1900.0,-83.11,FILLED,20210308 21:30:09,20210313 16:41:17
8,ETH_USDT,0.1414,1777.00,BUY,1900.0,-123.00,FILLED,20210308 18:37:37,20210313 16:41:17


In [12]:
#boolean dataframe from orders dataframe[ok]
my_columns = ['Symbol','B/S', 'Amount', 'Ticker Price', 'Momentum-Price',  'Momentum Profit', 'Status', 'Timestamp', 'Requisition Timestamp']
boo_dataframe = pd.DataFrame(columns = my_columns)

ttrades = nova_client.list_trades(symbol, limit=9)
tdata = ttrades['data']

api_url = f'https://api.novadax.com/v1/market/ticker?symbol={symbol}'
fdaty = requests.get(api_url).json()

fdat = nova_client.list_trades('BTC_BRL', limit = 10)

for symbol in symbols:
    #from 'finished' to 'filled'
    listorders = nova_client.list_orders(symbol, 'FILLED')
    s = 0
    for i in listorders['data'][:9]:
        ldata = listorders['data']
        ldc = ldata[s]
        amount = ldc['amount']
        price = float(ldc['price'])
        
        amount = float(ldc['amount'])
        cprice = float(amount) * float(price)
        edolar = bcr['USDT']
        eth_wal = float(bcr['ETH']) * float(price)
        edolarwal = float(bcr['USDT']) + float(eth_wal)
        bs = ldc['side']
        
        vxdata = tdata[s]
        vxi = float(vxdata['price'])
        momentump = float(price*2) - float(vxi)

        tspi = float(ldc['timestamp']/1000)
        tsp = datetime.fromtimestamp(tspi).strftime("%Y%m%d %H:%M:%S")
        status = ldc['status']
        
        localtimestamp = int(time())
        dl = localtimestamp
        tserver = datetime.fromtimestamp(dl).strftime("%Y%m%d %H:%M:%S")
        
        #boolean dataframe
        sell = 0
        buy = 0
        sm = 0
        bm = 0
        if bs == 'SELL':
            sell = float(price) - float(price * (0.03))  
            buy = momentump - float(momentump * (0.03))
        if bs == 'BUY':
            buy = float(price) - float(price * (0.03))
            sell = momentump - float(momentump * (0.03))
        profit = float(price*2/100)/1.00 + price
        bm = buy - momentump
        sm = sell - momentump
        if float(sublc) > 0.02:
            if vxi > price and amount < 0.02:
                #result = nova_client.create_order(symbol, 'LIMIT', 'SELL', price = profit, amount = amount)
                print("Sold ", amount, symbol, " for ", profit)
                #first block to be solved
            if vxi < price and amount < 0.02 or bs == 'SELL':
                print("Symbol not ready to sell")
        if float(sublc) < 0.02:
            if vxi < price and amount > 0.02 and bs == 'BUY ':
                #result = nova_client.create_order(symbol, 'LIMIT', 'BUY', price = math.floor(buy/1.00), amount = amount)
                print("The quantity bought was ETH: ", bm)
            if vxi < price and amount > 0.02 and bs == 'BUY':
                #result = nova_client.create_order(symbol, 'LIMIT', 'BUY', price = math.floor(buy/1.00), amount = amount)
                print("There was no balance.", amount, " was added for ", price)
                
        
        s = s + 1
        
        boo_dataframe.append(
                    pd.Series(
                    [
                        symbol,
                        bs,
                        amount,
                        price,
                        vxi,
                        momentump,
                        status,
                        tsp,
                        tserver,
                    ]
                    ,index = my_columns),
                ignore_index=True)

    
        boo_dataframe = boo_dataframe.append(
                pd.Series([
                        symbol,
                        bs,
                        amount,
                        price,
                        vxi,
                        momentump,
                        status,
                        tsp,
                        tserver,
                      ]
                ,index = my_columns), 
                ignore_index = True)

boo_dataframe.sort_values('Timestamp', ascending = False, inplace = True)
boo_dataframe.reset_index(drop = True, inplace = True)
boo_dataframe

,Symbol,B/S,Amount,Ticker Price,Momentum-Price,Momentum Profit,Status,Timestamp,Requisition Timestamp
0,ETH_USDT,SELL,0.0713,1820.00,1915.00,1725.00,FILLED,20210312 15:51:05,20210313 16:52:21
1,ETH_USDT,BUY,0.0714,1781.58,1907.32,1655.84,FILLED,20210312 15:50:02,20210313 16:52:21
2,ETH_USDT,SELL,0.0695,1830.37,1908.30,1752.44,FILLED,20210311 12:35:09,20210313 16:52:21
3,ETH_USDT,BUY,0.0697,1804.33,1910.03,1698.63,FILLED,20210310 17:46:17,20210313 16:52:21
4,ETH_USDT,SELL,0.0391,1850.77,1901.64,1799.90,FILLED,20210310 16:17:31,20210313 16:52:21
5,ETH_USDT,SELL,0.1405,1840.00,1908.43,1771.57,FILLED,20210309 11:50:06,20210313 16:52:21
6,ETH_USDT,BUY,0.1406,1825.00,1910.00,1740.00,FILLED,20210308 23:10:58,20210313 16:52:21
7,ETH_USDT,SELL,0.1413,1816.89,1900.00,1733.78,FILLED,20210308 21:30:09,20210313 16:52:21
8,ETH_USDT,BUY,0.1414,1777.00,1900.00,1654.00,FILLED,20210308 18:37:37,20210313 16:52:21


In [16]:
#boolean dataframe
sell = 0
buy = 0
sm = 0
bm = 0
if bs == 'SELL':
    sell = float(price) - float(price * (0.03))  
    buy = momentump - float(momentump * (0.03))
if bs == 'BUY':
    buy = float(price) - float(price * (0.03))
    sell = momentump - float(momentump * (0.03))
    profit = float(price*2/100)/1.00 + price
bm = buy - momentump
sm = sell - momentump
if float(sublc) > 0.02:
    if vxi > price and amount < 0.02:
        #result = nova_client.create_order(symbol, 'LIMIT', 'SELL', price = profit, amount = amount)
        print("Sold ", amount, symbol, " for ", profit)
                #first block to be solved
    if vxi < price and amount < 0.02 and bs == 'SELL':
        print("Symbol not ready to sell")
if float(sublc) < 0.02:
    if vxi < price and amount > 0.02 and bs == 'BUY ':
                #result = nova_client.create_order(symbol, 'LIMIT', 'BUY', price = math.floor(buy/1.00), amount = amount)
        print("The quantity bought was ETH: ", bm)
    if vxi < price and amount > 0.02 and bs == 'BUY':
                #result = nova_client.create_order(symbol, 'LIMIT', 'BUY', price = math.floor(buy/1.00), amount = amount)
        print("There was no balance.", amount, " was added for ", price)
                

BUY


In [ ]:
#P = S - B; SUM(Pi to Pn)  = Pi{(S - D) or (S - B)}[ok]
#if sell = true; [buy lower < sell][(sell - (contant*momentum)/2 or (buy + constant/momentum)/2[ok]

if vxi > price and amount > 0.02 :
    result = nova_client.create_order(symbol, 'LIMIT', 'SELL', price = profit, amount = amount)  

#first block to be solved
if vxi < price and amount < 0.02:
    print("Market is not ready to sell")

#if vxi < price and amount > 0.02:
    result = nova_client.create_order(symbol, 'LIMIT', 'BUY', price = bmprice, amount = amount)
    print("The quantity bought was ETH: ", bm)

#loop time count and do momentum profit auto cicle
#publish

### Format and export xlsx file
In the last section of the tbot a xlsx file is saved with the order table for later reading purposes.

In [ ]:
writer = pd.ExcelWriter('orders.xlsx', engine = 'xlsxwriter')
boo_dataframe.to_excel(writer, sheet_name = 'orderlist', index = False)

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [ ]:
column_formats = {
    'A':['Symbol', string_template],
    'B':['Buy/Sell', string_template],
    'C':['Amount', float_template],
    'D':['Last Orders Price', dollar_template],
    'F':['Momentum Price', ],
    'G':['Profit', ],
    'H':['Status', ],
    'I':['Timestamp', ],
    'J':['Requisition Timestamp', ],
    'K':['', ],
}
Symbol	B/S	Amount	Last Orders Price	Momentum Price	
Momentum Profit	Status	Timestamp	Requisition Timestamp
for column in column_formats.keys():
    writer.sheets['orderlist'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['orderlist'].write(f'{column}1', column_formats[column][0], column_formats[column][1])
   

In [ ]:
writer.save()